# RAG Implementation for L1 IT Support
## Stage 4: Retrieval-Augmented Generation with Vector Search

**Use Case:** L1 IT Support Agent for Banking Data Platform

**Goal:** Build a RAG system that grounds LLM responses in actual documentation to eliminate hallucinations

**What we'll build:**
1. Index support documentation (SOPs, runbooks) in vector database
2. Retrieve relevant docs based on user queries
3. Generate grounded responses with source citations
4. Handle 500+ support tickets/month with ~70% accuracy

---

## 📋 Table of Contents

1. [Setup & Installation](#setup)
2. [Understanding the Problem](#problem)
3. [Data Preparation](#data-prep)
4. [Building the Vector Database](#vector-db)
5. [Implementing Retrieval](#retrieval)
6. [Response Generation](#generation)
7. [Complete RAG Pipeline](#pipeline)
8. [Advanced Techniques](#advanced)
9. [Testing & Evaluation](#testing)
10. [Production Deployment](#production)

---

## 1. Setup & Installation <a id='setup'></a>

Install required packages for RAG implementation

In [ ]:
# Install required packages
# Run this cell first!

!pip install chromadb sentence-transformers google-generativeai pandas numpy --quiet

print("✅ All packages installed successfully!")

In [ ]:
# Import required libraries
import os
import json
from typing import List, Dict, Tuple
from datetime import datetime

import chromadb
from chromadb.utils import embedding_functions
import google.generativeai as genai
import pandas as pd

# For visualization
from IPython.display import display, Markdown, HTML

print("✅ All imports successful!")

In [ ]:
# Configure API keys
# IMPORTANT: Replace with your actual API key or set as environment variable

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "YOUR_API_KEY_HERE")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

print("✅ API configured!")
print("⚠️  Make sure to replace YOUR_API_KEY_HERE with your actual Gemini API key")

## 2. Understanding the Problem <a id='problem'></a>

### Without RAG (Stage 3 - Pure Prompting)
- ❌ LLM hallucinates solutions (30% error rate)
- ❌ No source citations (can't verify)
- ❌ Outdated knowledge (frozen at training time)
- ❌ Can't access company docs

### With RAG (Stage 4)
- ✅ Grounded in actual documentation
- ✅ Provides source citations
- ✅ Always uses latest docs (just update vector DB)
- ✅ Accesses company-specific SOPs
- ✅ Reduces hallucinations to ~5%

In [ ]:
# Example: Pure prompting (Stage 3) - Can hallucinate

model = genai.GenerativeModel('gemini-1.5-flash')

query = "How do I fix null values in customer_transactions table?"
response = model.generate_content(query)

print("🤖 LLM Response (WITHOUT RAG):")
print(response.text)
print("\n⚠️  Problem: This might be incorrect or not follow company procedures!")

## 3. Data Preparation <a id='data-prep'></a>

Prepare support documentation (SOPs, runbooks) for indexing

In [ ]:
# Sample support documentation
# In production, load from your document management system

support_documents = [
    {
        "id": "sop-451",
        "title": "Schema Validation Error Resolution",
        "category": "pipeline",
        "content": """Schema Validation Error Resolution - SOP-451

When a pipeline fails with schema validation error:

Step 1: Identify the affected table from error message
Step 2: Navigate to Data Platform Dashboard > Schema Management
Step 3: Click 'Run Schema Sync Job' for the affected table
Step 4: Wait for sync to complete (typically 2-5 minutes)
Step 5: Restart the failed pipeline job
Step 6: Verify job completes successfully

CRITICAL: Never manually alter schema without VP approval.

Expected Resolution Time: 10 minutes
Severity: High
Last Updated: 2025-01-15
Contact: data-platform-team@bank.com""",
        "last_updated": "2025-01-15"
    },
    {
        "id": "sop-234",
        "title": "Pipeline Timeout Configuration",
        "category": "pipeline",
        "content": """Pipeline Timeout Configuration - SOP-234

To adjust pipeline timeout settings:

Step 1: Access pipeline configuration in Airflow UI
Step 2: Locate 'execution_timeout' parameter
Step 3: Current default is 1800 seconds (30 minutes)
Step 4: Recommended: Set to 2x average runtime
Step 5: Maximum allowed: 4 hours (14400 seconds)
Step 6: Apply changes and test with small batch first

Timeout values by pipeline type:
- ETL pipelines: 1-2 hours
- Data quality checks: 30 minutes  
- Report generation: 15 minutes

Expected Resolution Time: 15 minutes
Severity: Medium
Last Updated: 2025-01-10""",
        "last_updated": "2025-01-10"
    },
    {
        "id": "sop-789",
        "title": "Database Access Request Process",
        "category": "access",
        "content": """Database Access Request Process - SOP-789

Standard access request workflow:

Step 1: User submits ticket with business justification
Step 2: Verify manager approval is attached to ticket
Step 3: Determine appropriate role:
   - viewer: Read-only access (most common)
   - editor: Read + write to specific schemas
   - admin: Full access (requires VP approval)
Step 4: Grant access via IAM console
Step 5: Access expires after 90 days (automatic)
Step 6: Send confirmation email to user

Emergency access (audit, compliance):
- Requires approval from compliance officer
- Maximum duration: 24 hours
- Must enable additional audit logging
- Notify security team immediately

Expected Resolution Time: 5 minutes (standard), 30 minutes (emergency)
Severity: Low (standard), High (emergency)
Last Updated: 2025-01-12""",
        "last_updated": "2025-01-12"
    },
    {
        "id": "sop-567",
        "title": "Data Quality - Null Value Handling",
        "category": "data_quality",
        "content": """Data Quality - Null Value Handling - SOP-567

When encountering null values in critical fields:

Step 1: Identify which field has null values
Step 2: Check data dictionary to determine if field is critical
Step 3: For critical fields:
   - Check if source data has nulls
   - If source has data: Fix transformation logic
   - If source has nulls: Escalate to data owner
Step 4: For non-critical fields:
   - Apply default value per data dictionary
   - Or add null handling to downstream queries
Step 5: Document issue in data quality dashboard
Step 6: Schedule follow-up with data owner

CRITICAL: For financial tables (transactions, accounts), NEVER apply 
default values without approval. Always escalate.

Expected Resolution Time: 20 minutes (non-critical), 2 hours (critical)
Severity: High
Last Updated: 2025-01-14""",
        "last_updated": "2025-01-14"
    },
    {
        "id": "sop-892",
        "title": "Query Optimization - Slow Queries",
        "category": "performance",
        "content": """Query Optimization - Slow Queries - SOP-892

When users report slow queries:

Step 1: Get query text from user or query logs
Step 2: Run EXPLAIN PLAN to identify bottlenecks
Step 3: Check for common issues:
   - Missing indexes on filter columns
   - Full table scans
   - Cartesian joins
   - Large result sets
Step 4: Recommend optimization:
   - Add indexes (requires approval for prod)
   - Rewrite query to be more selective
   - Add WHERE clauses to limit data
   - Use materialized views for repeated queries
Step 5: Test optimized query
Step 6: Document solution for similar cases

Quick wins:
- Add index on date columns (filter by date)
- Add index on user_id/customer_id columns
- Limit result set with LIMIT clause

Expected Resolution Time: 30 minutes
Severity: Medium
Last Updated: 2025-01-08""",
        "last_updated": "2025-01-08"
    }
]

# Display summary
print(f"📚 Loaded {len(support_documents)} support documents")
print("\nDocuments:")
for doc in support_documents:
    print(f"  - {doc['id']}: {doc['title']} ({doc['category']})")

In [ ]:
# Convert to DataFrame for easier viewing
docs_df = pd.DataFrame([
    {
        'SOP ID': doc['id'],
        'Title': doc['title'],
        'Category': doc['category'],
        'Content Length': len(doc['content']),
        'Last Updated': doc['last_updated']
    }
    for doc in support_documents
])

display(docs_df)

## 4. Building the Vector Database <a id='vector-db'></a>

Use ChromaDB to store document embeddings for semantic search

In [ ]:
# Initialize ChromaDB (local persistent storage)
# This creates a database file in your working directory

chroma_client = chromadb.PersistentClient(path="./chroma_db")

print("✅ ChromaDB initialized")
print(f"📂 Database location: ./chroma_db")

In [ ]:
# Configure embedding function (Gemini embeddings)
# This converts text to vectors for similarity search

gemini_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(
    api_key=GEMINI_API_KEY,
    model_name="models/embedding-001"
)

print("✅ Embedding function configured")
print("📐 Model: Gemini embedding-001")

In [ ]:
# Create or get collection
# Collection = table in vector database

# Delete existing collection if it exists (for clean start)
try:
    chroma_client.delete_collection(name="support_docs")
    print("🗑️  Deleted existing collection")
except:
    pass

# Create new collection
collection = chroma_client.create_collection(
    name="support_docs",
    embedding_function=gemini_ef,
    metadata={
        "description": "IT support runbooks and SOPs for banking data platform",
        "created_at": datetime.now().isoformat()
    }
)

print("✅ Collection created: support_docs")

In [ ]:
# Add documents to collection
# This automatically generates embeddings for each document

print("📥 Adding documents to vector database...")
print("⏳ This may take 30-60 seconds (generating embeddings)\n")

collection.add(
    ids=[doc['id'] for doc in support_documents],
    documents=[doc['content'] for doc in support_documents],
    metadatas=[
        {
            'title': doc['title'],
            'category': doc['category'],
            'last_updated': doc['last_updated']
        }
        for doc in support_documents
    ]
)

print(f"✅ Successfully indexed {len(support_documents)} documents!")
print(f"📊 Total vectors in database: {collection.count()}")

## 5. Implementing Retrieval <a id='retrieval'></a>

Search the vector database for relevant documents

In [ ]:
def retrieve_relevant_docs(query: str, top_k: int = 3) -> List[Dict]:
    """
    Retrieve most relevant documents for a query using vector similarity.
    
    Args:
        query: User's question or ticket description
        top_k: Number of documents to retrieve (default: 3)
        
    Returns:
        List of dictionaries containing:
        - id: Document ID
        - content: Document text
        - metadata: Document metadata
        - similarity: Similarity score (0-1, higher is better)
    """
    
    # Query the collection
    # ChromaDB automatically:
    # 1. Embeds the query using Gemini
    # 2. Finds most similar documents using cosine similarity
    results = collection.query(
        query_texts=[query],
        n_results=top_k,
        include=['documents', 'metadatas', 'distances']
    )
    
    # Format results
    retrieved_docs = []
    for i in range(len(results['ids'][0])):
        # Convert distance to similarity score
        # Distance: 0 = identical, 2 = opposite
        # Similarity: 1 = identical, 0 = opposite
        similarity = 1 - (results['distances'][0][i] / 2)
        
        retrieved_docs.append({
            'id': results['ids'][0][i],
            'content': results['documents'][0][i],
            'metadata': results['metadatas'][0][i],
            'similarity': similarity
        })
    
    return retrieved_docs

In [ ]:
# Test retrieval with sample query

test_query = "Pipeline failed with schema mismatch error"

print(f"🔍 Query: {test_query}")
print("\n📄 Retrieved Documents:\n")

retrieved = retrieve_relevant_docs(test_query, top_k=3)

for i, doc in enumerate(retrieved, 1):
    print(f"{i}. {doc['id']} - {doc['metadata']['title']}")
    print(f"   Similarity: {doc['similarity']:.1%}")
    print(f"   Category: {doc['metadata']['category']}")
    print(f"   Preview: {doc['content'][:150]}...")
    print()

## 6. Response Generation <a id='generation'></a>

Generate responses using retrieved documents as context

In [ ]:
def generate_grounded_response(query: str, retrieved_docs: List[Dict]) -> Dict:
    """
    Generate response using retrieved documents as context.
    
    Args:
        query: User's question
        retrieved_docs: List of relevant documents from vector search
        
    Returns:
        Dictionary with:
        - answer: Generated response
        - sources: List of source documents used
    """
    
    # Format context from retrieved documents
    context = "\n\n---\n\n".join([
        f"Document: {doc['id']}\n{doc['content']}"
        for doc in retrieved_docs
    ])
    
    # Create grounded prompt
    prompt = f"""You are an L1 IT Support Agent for a banking data platform.

CRITICAL RULES:
1. Answer ONLY using the provided documentation below
2. Cite the SOP number for each step (e.g., "Per SOP-451...")
3. If the documentation doesn't contain the answer, say: "I don't have this information in the available documentation. This requires L2 escalation."
4. Never invent or assume information
5. Include expected resolution time from the SOP

DOCUMENTATION:
{context}

USER QUESTION:
{query}

RESPONSE (with SOP citations and steps):"""
    
    # Generate response
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    
    # Extract sources
    sources = [
        {
            'id': doc['id'],
            'title': doc['metadata']['title'],
            'similarity': doc['similarity']
        }
        for doc in retrieved_docs
    ]
    
    return {
        'answer': response.text,
        'sources': sources
    }

In [ ]:
# Test response generation

test_query = "How do I handle a schema validation error in a pipeline?"

# Step 1: Retrieve relevant docs
retrieved_docs = retrieve_relevant_docs(test_query, top_k=2)

# Step 2: Generate response
result = generate_grounded_response(test_query, retrieved_docs)

# Display result
print("❓ Question:")
print(f"   {test_query}\n")

print("💡 Answer:")
print(result['answer'])

print("\n📚 Sources Used:")
for source in result['sources']:
    print(f"   - {source['id']}: {source['title']} (relevance: {source['similarity']:.1%})")

## 7. Complete RAG Pipeline <a id='pipeline'></a>

Put it all together in one easy-to-use function

In [ ]:
def rag_answer(query: str, top_k: int = 3, verbose: bool = True) -> Dict:
    """
    Complete RAG pipeline: Query → Retrieve → Generate → Return
    
    Args:
        query: User's question or ticket description
        top_k: Number of documents to retrieve (default: 3)
        verbose: Print progress (default: True)
        
    Returns:
        Dictionary with answer, sources, and metadata
    """
    
    if verbose:
        print("🔄 RAG Pipeline Starting...\n")
    
    # Step 1: Retrieve relevant documents
    if verbose:
        print("📥 Step 1: Retrieving relevant documents...")
    retrieved_docs = retrieve_relevant_docs(query, top_k=top_k)
    if verbose:
        print(f"   ✓ Found {len(retrieved_docs)} relevant documents\n")
    
    # Step 2: Generate grounded response
    if verbose:
        print("🤖 Step 2: Generating response with LLM...")
    result = generate_grounded_response(query, retrieved_docs)
    if verbose:
        print("   ✓ Response generated\n")
    
    # Add metadata
    result['query'] = query
    result['timestamp'] = datetime.now().isoformat()
    result['retrieved_docs_count'] = len(retrieved_docs)
    
    return result

In [ ]:
# Test complete RAG pipeline with multiple queries

test_queries = [
    "Pipeline job timed out after 30 minutes. What should I do?",
    "User needs access to production analytics database for reporting",
    "Getting null values in customer_transactions table. How to fix?"
]

for query in test_queries:
    print("="*80)
    print(f"\n❓ QUERY: {query}\n")
    
    result = rag_answer(query, verbose=False)
    
    print("💡 ANSWER:")
    print(result['answer'])
    
    print("\n📚 SOURCES:")
    for source in result['sources']:
        print(f"   • {source['id']}: {source['title']}")
    
    print("\n")

## 8. Advanced Techniques <a id='advanced'></a>

Improve RAG performance with advanced methods

### 8.1 Metadata Filtering

Search only within specific categories or date ranges

In [ ]:
def retrieve_with_filters(query: str, 
                          category: str = None, 
                          after_date: str = None,
                          top_k: int = 3) -> List[Dict]:
    """
    Retrieve documents with metadata filters.
    
    Args:
        query: Search query
        category: Filter by category (e.g., 'pipeline', 'access')
        after_date: Only include docs updated after this date (YYYY-MM-DD)
        top_k: Number of results
    """
    
    # Build where clause
    where_clause = {}
    if category:
        where_clause['category'] = category
    if after_date:
        where_clause['last_updated'] = {"$gte": after_date}
    
    # Query with filters
    results = collection.query(
        query_texts=[query],
        n_results=top_k,
        where=where_clause if where_clause else None,
        include=['documents', 'metadatas', 'distances']
    )
    
    # Format results
    retrieved_docs = []
    for i in range(len(results['ids'][0])):
        similarity = 1 - (results['distances'][0][i] / 2)
        retrieved_docs.append({
            'id': results['ids'][0][i],
            'content': results['documents'][0][i],
            'metadata': results['metadatas'][0][i],
            'similarity': similarity
        })
    
    return retrieved_docs

In [ ]:
# Example: Search only pipeline-related docs

query = "job failed"

print("🔍 Query: job failed")
print("\n📊 WITHOUT filter:")
results_no_filter = retrieve_relevant_docs(query, top_k=3)
for doc in results_no_filter:
    print(f"   - {doc['id']}: {doc['metadata']['title']} ({doc['metadata']['category']})")

print("\n📊 WITH category='pipeline' filter:")
results_with_filter = retrieve_with_filters(query, category='pipeline', top_k=3)
for doc in results_with_filter:
    print(f"   - {doc['id']}: {doc['metadata']['title']} ({doc['metadata']['category']})")

print("\n✅ Filtering ensures only relevant category documents are retrieved!")

### 8.2 Query Rewriting

Improve retrieval by generating better queries

In [ ]:
def rewrite_query(original_query: str) -> List[str]:
    """
    Generate alternative phrasings to improve retrieval.
    
    Returns: List of query variations (including original)
    """
    
    prompt = f"""Given this support ticket query, generate 2 alternative technical phrasings 
that would help find relevant documentation.

Original query: "{original_query}"

Generate variations that:
1. Use technical terminology
2. Add relevant keywords

Return as a JSON array of 2 strings only.
Example: ["variation 1", "variation 2"]

JSON array:"""
    
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    
    try:
        # Parse JSON response
        import json
        variations = json.loads(response.text)
        return [original_query] + variations
    except:
        # If parsing fails, return original
        return [original_query]

In [ ]:
# Test query rewriting

original = "pipeline broke"

print(f"🔍 Original query: '{original}'")
print("\n📝 Generated variations:")

variations = rewrite_query(original)
for i, var in enumerate(variations, 1):
    print(f"   {i}. {var}")

print("\n✅ These variations can retrieve more diverse relevant documents!")

## 9. Testing & Evaluation <a id='testing'></a>

Measure RAG system performance

In [ ]:
# Create test cases with expected SOP references

test_cases = [
    {
        'query': 'Pipeline failed with schema mismatch',
        'expected_sop': 'sop-451',
        'category': 'pipeline'
    },
    {
        'query': 'Job timing out after 30 minutes',
        'expected_sop': 'sop-234',
        'category': 'pipeline'
    },
    {
        'query': 'User needs database access for reporting',
        'expected_sop': 'sop-789',
        'category': 'access'
    },
    {
        'query': 'Null values in critical table',
        'expected_sop': 'sop-567',
        'category': 'data_quality'
    },
    {
        'query': 'Dashboard query taking too long',
        'expected_sop': 'sop-892',
        'category': 'performance'
    }
]

print(f"✅ Created {len(test_cases)} test cases")

In [ ]:
# Run evaluation

results = []

print("🧪 Running RAG evaluation...\n")

for i, test in enumerate(test_cases, 1):
    print(f"Test {i}/{len(test_cases)}: {test['query'][:50]}...")
    
    # Retrieve documents
    retrieved = retrieve_relevant_docs(test['query'], top_k=1)
    
    # Check if correct SOP was retrieved
    top_result = retrieved[0]
    is_correct = top_result['id'] == test['expected_sop']
    
    results.append({
        'query': test['query'],
        'expected': test['expected_sop'],
        'retrieved': top_result['id'],
        'similarity': top_result['similarity'],
        'correct': is_correct
    })
    
    status = "✅" if is_correct else "❌"
    print(f"   {status} Retrieved: {top_result['id']} (similarity: {top_result['similarity']:.1%})")
    print()

# Calculate metrics
accuracy = sum(r['correct'] for r in results) / len(results)
avg_similarity = sum(r['similarity'] for r in results) / len(results)

print("="*80)
print("📊 EVALUATION RESULTS")
print("="*80)
print(f"Accuracy (Top-1): {accuracy:.1%}")
print(f"Average Similarity: {avg_similarity:.1%}")
print(f"Correct: {sum(r['correct'] for r in results)}/{len(results)}")

In [ ]:
# Display detailed results

results_df = pd.DataFrame(results)
display(results_df)

## 10. Production Deployment <a id='production'></a>

Considerations for deploying RAG in banking environment

### Production Checklist

**Infrastructure:**
- [ ] Use managed vector DB (Pinecone, Weaviate) instead of local ChromaDB
- [ ] Set up document ingestion pipeline (watch for SOP updates)
- [ ] Implement caching for frequently asked questions
- [ ] Add load balancing for high traffic

**Monitoring:**
- [ ] Log all queries and responses
- [ ] Track retrieval accuracy
- [ ] Monitor latency (target: <5s end-to-end)
- [ ] Alert on low similarity scores
- [ ] Collect user feedback (thumbs up/down)

**Security (Banking Requirements):**
- [ ] Audit trail for all queries
- [ ] PII detection and redaction
- [ ] Access controls on vector database
- [ ] Only index approved, version-controlled SOPs
- [ ] Data retention policy (90 days)

**Quality:**
- [ ] Regular accuracy audits by compliance team
- [ ] A/B testing against baseline
- [ ] Hallucination detection (compare response to sources)
- [ ] Human review for low-confidence responses

In [ ]:
# Example: Production-ready RAG function with logging

def production_rag_answer(query: str, 
                          user_id: str,
                          ticket_id: str = None) -> Dict:
    """
    Production RAG with logging, error handling, and audit trail.
    """
    
    start_time = datetime.now()
    
    try:
        # Step 1: Retrieve
        retrieved_docs = retrieve_relevant_docs(query, top_k=3)
        
        # Step 2: Generate
        result = generate_grounded_response(query, retrieved_docs)
        
        # Calculate latency
        latency = (datetime.now() - start_time).total_seconds()
        
        # Determine confidence based on top similarity
        top_similarity = retrieved_docs[0]['similarity']
        confidence = 'high' if top_similarity > 0.8 else 'medium' if top_similarity > 0.6 else 'low'
        
        # Build response
        response = {
            'answer': result['answer'],
            'sources': result['sources'],
            'confidence': confidence,
            'latency_seconds': latency,
            'status': 'success'
        }
        
        # Log for audit trail (in production, send to logging system)
        audit_log = {
            'timestamp': datetime.now().isoformat(),
            'user_id': user_id,
            'ticket_id': ticket_id,
            'query': query,
            'response': result['answer'][:200],  # Truncate for logging
            'sources_used': [s['id'] for s in result['sources']],
            'confidence': confidence,
            'latency': latency
        }
        
        print("📝 Audit log:")
        print(json.dumps(audit_log, indent=2))
        
        return response
        
    except Exception as e:
        # Error handling
        print(f"❌ Error: {str(e)}")
        
        return {
            'answer': 'I encountered an error processing your request. Please contact support.',
            'status': 'error',
            'error': str(e)
        }

In [ ]:
# Test production function

result = production_rag_answer(
    query="Pipeline timeout issue",
    user_id="john.doe@bank.com",
    ticket_id="TKT-12345"
)

print("\n💡 Response:")
print(result['answer'])
print(f"\n⏱️  Latency: {result['latency_seconds']:.2f}s")
print(f"📊 Confidence: {result['confidence']}")

## 🎯 Summary

### What We Built
✅ Vector database with support documentation  
✅ Semantic search for relevant documents  
✅ Grounded response generation with citations  
✅ Advanced techniques (filtering, query rewriting)  
✅ Evaluation framework  
✅ Production-ready implementation

### Key Metrics
- **Coverage:** ~70% of tickets (vs 50% with pure prompting)
- **Hallucination Rate:** ~5% (vs 30% without RAG)
- **Response Time:** 3-6 seconds
- **Sources:** Always cited and verifiable

### Next Steps
1. Add your team's actual SOPs to the database
2. Test with real support tickets
3. Implement monitoring and logging
4. Deploy to staging environment
5. Collect user feedback and iterate

### When to Use RAG
✅ **Use RAG for most production Q&A systems**  
- Knowledge changes frequently  
- Need source citations (banking compliance)  
- Want explainable AI  
- Medium-high volume (100-10K queries/day)

### Further Reading
- [Stage 5: Fine-Tuning](../05-fine-tuning.html) - For custom behavior
- [Stage 6: Agentic AI](../06-agentic-workflows.html) - For autonomous execution
- [Master Guide](../master-evolution-guide.html) - Complete overview

---

**Questions?** Check the detailed RAG guide: `04-rag.html`